In [124]:
from bs4 import BeautifulSoup as bs
import re
from dateutil.parser import parse
# from datetime import datetime
import pandas as pd
import json
import sys

In [11]:
def parse_html(path): 
    with open(path) as f:
        # read file
        content = f.read()
        # parse with bs
        soup = bs(content, 'lxml')

    return soup

In [12]:
watch_history = parse_html('../Rådata/Takeout/YouTube and YouTube Music/history/watch-history.html')

In [13]:
def is_a_video(tag): 
    '''
    Filter to remove ads and videos that still ex
    takes a block of html code and returns a boolean with whether this is an ad or not
    '''
    not_an_ad = not(bool(re.search('From Google Ads', tag.get_text())))
    not_deleted = len(tag.find_all('a')) > 2
    return (not_an_ad & not_deleted)
       
 

In [14]:
def parse_watch_history(soup):
    '''takes a parsed tree of watch history (wh) from beautiful soup and returns a simple df with the relevant data'''
    # hver yderste blok har klassen 'outer-cell mdl-cell mdl-cell--12-col mdl-shadow--2dp' - Her henter vi dem alle sammen
    blocks = soup.find_all(class_ = 'outer-cell mdl-cell mdl-cell--12-col mdl-shadow--2dp')
    blocks_cleaned = [block for block in blocks if is_a_video(block)]
    print(f'{len(blocks)} entries; {len(blocks_cleaned)} after cleaning. {len(blocks) - len(blocks_cleaned)} ads and deleted videos removed!')
    return blocks_cleaned

watch_history_blocks = parse_watch_history(watch_history)

30345 entries; 28332 after cleaning. 2013 ads and deleted videos removed!


In [15]:
# Finding a video and an add for testing! 
video = watch_history_blocks[0]
ad =  watch_history_blocks[5]

# To create the dataset! 

Det store problem her er at få datoen til at blive hentet rigtigt fra teksten solvom der kan være overlap med titlen af kanalen (hvis den slutter på et tal) eller hvis den har en mærkelig sætning der siger hvornår den er set på - det kan forekomme i nogle tilfælde. 

Den foreløbige løsning er kombinationen af f og r strenge - her kan variable puttes ind i en regex. Det er vigtigt at huske på at kvantifiers i regex '{}' så skal have et dobbelt sæt klammer rund om sig!! 

In [16]:
# All of this is to find out how to do the cleansing properly - it should work now but it is here for posterity in case it needs fixing! 
watched_at_videos = [video for video in watch_history_blocks if re.search(r'Watched at \d\d:\d\d', video.text)]

In [17]:
watched_at_video = watched_at_videos[0]
watched_at_video.text
match = re.search(r'Watched at \d\d:\d\d', watched_at_video.text)
match.group()

'Watched at 22:16'

In [18]:
# All of this is to find out how to do the cleansing properly - it should work now but it is here for posterity in case it needs fixing! 

# match = re.search(watch_history_blocks[0].text, r'\\n\d?\d \w{3,4} \d{4}, \d{2}:\d{2}:\d{2}[^P]*')
# print(match)
# watch_history_blocks[0].find(class_ = 'content-cell mdl-cell mdl-cell--6-col mdl-typography--body-1').text
block = watched_at_video
print(block.text)
links = block.find_all('a')

channel_title = re.escape(links[1].text)
watched_at = re.search(r'Watched at \d\d:\d\d', block.text) # Watched at 22:16

if watched_at:
   search_string = fr'{channel_title}{watched_at.group()}(\d?\d \w{{3,4}} \d{{4}}, \d{{2}}:\d{{2}}:\d{{2}}[^P]*)'  
else:
    search_string = fr'{channel_title}(\d?\d \w{{3,4}} \d{{4}}, \d{{2}}:\d{{2}}:\d{{2}}[^P]*)' 

print(search_string)

match = re.search(search_string, block.text)

print(search_string, match)
match[1]

YouTubeWatched YouTube Music Ableton Walkthroughs: Part 1Red Means RecordingWatched at 22:169 Nov 2019, 23:05:11 CESTProducts: YouTubeWhy is this here? This activity was saved to your Google Account because the following settings were on: YouTube watch history. You can control these settings  here.
Red\ Means\ RecordingWatched at 22:16(\d?\d \w{3,4} \d{4}, \d{2}:\d{2}:\d{2}[^P]*)
Red\ Means\ RecordingWatched at 22:16(\d?\d \w{3,4} \d{4}, \d{2}:\d{2}:\d{2}[^P]*) <re.Match object; span=(57, 117), match='Red Means RecordingWatched at 22:169 Nov 2019, 23>


'9 Nov 2019, 23:05:11 CEST'

In [19]:
video_titles = []
video_links = []
channel_titles = []
channel_links = []
date_watched = []

# i = 0 # debugging! 
for block in watch_history_blocks:
    links = block.find_all('a')
    
    video_titles.append(links[0].text)
    video_links.append(links[0]['href'])
    channel_title = links[1].text # get channel title for date search! 
    channel_titles.append(channel_title)
    channel_links.append(links[1]['href'])

    # Useful for debugging
    # i += 1
    # print(f'{len(links)} links; entry {i} with title {links[0].text}')

    # find out if text has the weird watched at text so we can avoid it! 
    watched_at = re.search(r'Watched at \d\d:\d\d', block.text) 
    channel_title = re.escape(channel_title) # escape so we avoid problems with channel titles full of weird characters! 

    if watched_at:
        search_string = fr'{channel_title}{watched_at.group()}(\d?\d \w{{3,4}} \d{{4}}, \d{{2}}:\d{{2}}:\d{{2}}[^P]*)'  
    else:
        search_string = fr'{channel_title}(\d?\d \w{{3,4}} \d{{4}}, \d{{2}}:\d{{2}}:\d{{2}}[^P]*)' 

    date_string = re.search(search_string, block.text)

    if watched_at:
        date_watched.append(parse(date_string.group(1)))
    else:
        date_watched.append(parse(date_string.group(1)))


In [20]:
# making sure that we only get the ids! 
for video in video_links:
    match = re.search(r'https://www.youtube.com/watch\?v=(.*)', video)
    if not match:
        print(video)

In [21]:
type(date_watched[0])

datetime.datetime

In [22]:
date_string = re.search(r'\d?\d \w{3} \d{4}, \d{2}:\d{2}:\d{2}[^P]*', video.text)


AttributeError: 'str' object has no attribute 'text'

In [ ]:
date = parse(date_string[0])
date_string[0]

'9 Aug 2022, 13:27:58 CEST'

In [ ]:
print(date.isoformat())
print(pd.Timestamp(date))

2022-08-09T13:27:58+02:00
2022-08-09 13:27:58+02:00


In [ ]:
# dataframe = pd.DataFrame({
#     'video_title' : video_titles,
#     'video_link' : video_links,
#     'channel_title' : channel_titles,
#     'channel_link' : channel_links,
#     'date_watched' : date_watched
# })
dataframe = pd.read_csv('../Renset data/watch_history_df.csv', index_col=0, parse_dates=['date_watched'])

In [116]:
print(dataframe.dtypes)
dataframe.head()
# dataframe.shape


video_title      object
video_id         object
channel_title    object
channel_link     object
date_watched     object
dtype: object


,video_title,video_id,channel_title,channel_link,date_watched
0,Schrödinger's Levels - Astortion Devlog,oPpzT6JKGk0,aarthificial,https://www.youtube.com/channel/UCtEwVJZABCd0t...,2022-08-09 13:27:58+02:00
1,John mayer - neon (after 2 years of practice),dfjTp8YdNgk,אביה מילר,https://www.youtube.com/channel/UC-z1y3pCBSgTT...,2022-08-09 13:27:39+02:00
2,Are Chinese watches worth it?,HljYyn8U4a0,Talking Timepieces,https://www.youtube.com/channel/UCQkmGbeY6QVgw...,2022-08-09 13:13:38+02:00
3,An Intricate $180 Beauty! | The Sugess Moonpha...,O50euB79Jjc,Tone,https://www.youtube.com/channel/UCn0vfLJzUsCFu...,2022-08-09 09:46:47+02:00
4,Corgeut Seamaster 300 Spectre v Omega Seamaste...,93v5Bso2NDU,Wonder of Watches,https://www.youtube.com/channel/UCCPbvLaMx8P-c...,2022-08-09 09:45:38+02:00


In [ ]:
dataframe.rename(columns={'video_link' : 'video_id'}, inplace = True)

In [ ]:
dataframe.loc[0, 'date_watched'].isoformat()
# dataframe.shape

'2022-08-09T13:27:58+02:00'

# Hent beskrivelse af video fra api

In [ ]:
# new imports!
import googleapiclient.discovery
import google_auth_oauthlib.flow

In [144]:
df = pd.read_csv('../Renset data/mindre_df.csv', index_col=0)

In [139]:
df = df.rename(columns={'video_link': 'video_id'})

In [146]:
# df.to_csv('../Renset data/watch_history_df.csv')
df.shape

(1000, 5)

In [121]:
def get_authenticated_service(credential_path):
    api_service_name = "youtube"
    api_version = "v3"

    client_secrets_file = credential_path
    scopes = ["https://www.googleapis.com/auth/youtube.force-ssl"]

    # API client
    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
        client_secrets_file, scopes)

    credentials = flow.run_local_server(host='localhost',
                                        port=8080,
                                        authorization_prompt_message='Please visit this URL: {url}',
                                        success_message='The auth flow is complete; you may close this window.',
                                        open_browser=True)

    return (googleapiclient.discovery.build(api_service_name,
                                            api_version,
                                            credentials=credentials))


In [122]:
credentials = [
    '../credentials/credentials1.json',
    '../credentials/credentials2.json',
    '../credentials/credentials3.json'
]

current_credential = 0
attempts = 0

def get_video_info(authenticated_service, id): 
    request = authenticated_service.videos().list(
        part='snippet',
        id=id,
        fields='items(snippet(description)),items(snippet(thumbnails(high(url)))),\
        items(snippet(publishedAt)),items(snippet(tags))items(snippet(channelId))'
    )

    # return request.execute()
    try: 
        return request.execute()

    except googleapiclient.errors.HttpError as e:  
        print(e)
        if e.error_details[0]['reason'] == 'quotaExceeded':
            current_credential += 1 
            if current_credential >= len(credentials) - 1:
                current_credential = 0

            youtube = get_authenticated_service(credentials[current_credential])
            
            print(f'Having trouble with {id=}, trying credential {current_credential}')
            get_video_info(youtube, id)

In [126]:
def parse_video_response(response):
    '''
    retrieves video info from a response object and appends it to the lists
    holding all the data
    '''
    response = response['items'][0]['snippet']  # gets only the snippet dict!
    description = response.get('description', pd.NA)
    channel_id = response.get('channelId', pd.NA)
    published_at = response.get('publishedAt', pd.NA)
    thumbnail = response.get(
        'thumbnails', pd.NA
    ).get(
        'high', pd.NA
    ).get(
        'url', pd.NA
    )
    tag = response.get('tags', pd.NA)

    return description, channel_id, published_at, thumbnail, tag

In [149]:

def main():
    channel_ids = []
    published_at = []
    descriptions = []
    thumbnails = []
    tags = []
    removed_videos_ids = []

    # if len(sys.argv) < 1:
    #     raise SystemExit(f"Usage: {sys.argv[0]} <path_to_df>")

    # path_to_df = sys.argv[1]

    df = pd.read_csv('../Renset data/mindre_df.csv', index_col=0)

    youtube = get_authenticated_service('../credentials/credentials1.json')

    for i, id in enumerate(df.video_id):
        response = get_video_info(youtube, id)
        if response['items']:
            description, channel_id, published, thumbnail, tag = parse_video_response(response)
            channel_ids.append(channel_id)
            published_at.append(published)
            descriptions.append(description)
            thumbnails.append(thumbnail)
            tags.append(tag)
        else:
            removed_videos_ids.append(id)
            print(f'Video number {i=} with {id=} did not have any information!j')

    # fjern de id'er der ikke kan fås information på
    full_df = df[~df['video_id'].isin(removed_videos_ids)]
    # tilføj de nye lister som kolonner i en samlet df
    full_df = full_df.assign(
        channel_id = channel_ids,
        published_date = published_at,
        description = descriptions,
        thumbnail = thumbnails,
        tag = tags
    )

    full_df.to_csv('../Renset data/history_w_videoinfo.csv')
 
if __name__ == "__main__":
   main()
   


Please visit this URL: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=859790329153-3ncuv1l9tuhpb0nocncthbhbiv62r7jm.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=73aV6JIqrLG8z6ua9VkQp2ziIp8qF7&access_type=offline
Video number i=941 with id='ZfPmBMGXXzY' did not have any information!j


In [151]:
df = pd.read_csv('../Renset data/history_w_videoinfo.csv', index_col = 0)
df.head()

,Unnamed: 0,video_title,video_id,channel_title,channel_link,date_watched,channel_id,published_date,description,thumbnail,tag
0,0,Schrödinger's Levels - Astortion Devlog,oPpzT6JKGk0,aarthificial,https://www.youtube.com/channel/UCtEwVJZABCd0t...,2022-08-09 13:27:58+02:00,UCtEwVJZABCd0tels2KIpKGQ,2022-07-30T14:59:48Z,=== Wishlist Astortion on Steam ===\nhttps://s...,https://i.ytimg.com/vi/oPpzT6JKGk0/hqdefault.jpg,"['game dev', 'devlog', 'dev log', 'programming..."
1,1,John mayer - neon (after 2 years of practice),dfjTp8YdNgk,אביה מילר,https://www.youtube.com/channel/UC-z1y3pCBSgTT...,2022-08-09 13:27:39+02:00,UC-z1y3pCBSgTT30ft7BVmRQ,2022-06-26T18:23:37Z,NaN,https://i.ytimg.com/vi/dfjTp8YdNgk/hqdefault.jpg,NaN
2,2,Are Chinese watches worth it?,HljYyn8U4a0,Talking Timepieces,https://www.youtube.com/channel/UCQkmGbeY6QVgw...,2022-08-09 13:13:38+02:00,UCQkmGbeY6QVgwUyIzJ6LbIg,2021-03-03T13:30:50Z,In this episode of #TalkingTimepieces​ we look...,https://i.ytimg.com/vi/HljYyn8U4a0/hqdefault.jpg,NaN
3,3,An Intricate $180 Beauty! | The Sugess Moonpha...,O50euB79Jjc,Tone,https://www.youtube.com/channel/UCn0vfLJzUsCFu...,2022-08-09 09:46:47+02:00,UCn0vfLJzUsCFuRwEiBf7TSw,2021-09-15T01:17:24Z,"Good day all, in this video, I showcase the Br...",https://i.ytimg.com/vi/O50euB79Jjc/hqdefault.jpg,"['watch', 'watches', 'moonphase', 'cheap watch..."
4,4,Corgeut Seamaster 300 Spectre v Omega Seamaste...,93v5Bso2NDU,Wonder of Watches,https://www.youtube.com/channel/UCCPbvLaMx8P-c...,2022-08-09 09:45:38+02:00,UCCPbvLaMx8P-ccHDKWQ2agw,2020-08-07T22:33:58Z,Corgeut Seamaster 300 Homage Review\n\nToday I...,https://i.ytimg.com/vi/93v5Bso2NDU/hqdefault.jpg,"['Corgeut', 'corgeut seamaster', 'corgeut seam..."


# Hent captions via tredjepartsscript

In [90]:
from youtube_transcript_api import YouTubeTranscriptApi

In [91]:
test_data = dataframe.iloc[5]

YouTubeTranscriptApi.get_transcript(test_data.video_id, languages=['da', 'en'])


[{'text': 'hello and welcome to an episode of i',
  'start': 0.399,
  'duration': 3.201},
 {'text': 'bought a watch', 'start': 2.56, 'duration': 3.92},
 {'text': 'we are storming ahead with subscribers',
  'start': 3.6,
  'duration': 3.279},
 {'text': 'but', 'start': 6.48, 'duration': 1.84},
 {'text': "what i'm really enjoying is the",
  'start': 6.879,
  'duration': 3.601},
 {'text': 'engagement in the comments section',
  'start': 8.32,
  'duration': 3.92},
 {'text': 'i love that people are giving their own',
  'start': 10.48,
  'duration': 4.079},
 {'text': 'takes on the watches as well as opinions',
  'start': 12.24,
  'duration': 4.24},
 {'text': "if they've bought similar pieces because",
  'start': 14.559,
  'duration': 3.121},
 {'text': 'at the end of the day', 'start': 16.48, 'duration': 3.44},
 {'text': 'this just gives other viewers a better',
  'start': 17.68,
  'duration': 3.759},
 {'text': "idea of what they're getting", 'start': 19.92, 'duration': 3.599},
 {'text': 'and 

# Døde ideers land! 

In [5]:
# new imports! 
import googleapiclient.discovery
import google_auth_oauthlib.flow

In [51]:
# points used for downloading all my data
#  
# points_pr_day = 10000
# n_videos = 28332
# caption_points = 200 * n_videos
# video_info = 0 * n_videos

# points = caption_points + video_info
# print(f'points used for getting all captions: {points:.1E}, \n\
# points in a day: {points_pr_day:.1E}')


points used for getting all captions: 5.7E+06, 
points in a day: 1.0E+04


In [6]:
with open('../credentials.json') as f:
    credentials = json.load(f)

In [80]:
test_data

video_title      Corgeut "Spectre" 007 (Seamaster 300 Homage Re...
video_id                                               9ZA_VTmtKTo
channel_title                                     I Bought A Watch
channel_link     https://www.youtube.com/channel/UCxeqkrP0X2CaS...
date_watched                             2022-08-09 09:29:18+02:00
Name: 5, dtype: object

In [79]:
test_data = dataframe.iloc[5]
ids = test_data.video_id

In [ ]:
def get_authenticated_service():
    '''
    Function to authorize the use of ones authorization to gather data! 
    '''

In [ ]:
def get_caption(video_id):
    '''
    Takes a video ID and retrieves the caption from YouTubes API
    '''
    pass

In [81]:
# this is going to be rewritten into get authenticated service! 

api_service_name = "youtube"
api_version = "v3"

ids = test_data['video_id']
client_secrets_file = '../credentials.json'
scopes = ["https://www.googleapis.com/auth/youtube.force-ssl"]

# API client
flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
    client_secrets_file, scopes)

credentials = flow.run_local_server(host='localhost',
                                    port=8080,
                                    authorization_prompt_message='Please visit this URL: {url}',
                                    success_message='The auth flow is complete; you may close this window.',
                                    open_browser=True)

youtube = googleapiclient.discovery.build(
    api_service_name, api_version, credentials=credentials)


Please visit this URL: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=859790329153-3ncuv1l9tuhpb0nocncthbhbiv62r7jm.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=8zePmoerJgWAEchxqKrYF05z2J2BL5&access_type=offline


In [ ]:
with open('../credentials.json') as f:
    credentials = json.load(f)

In [76]:
# wrong_response = {
#     'items': [{
#         'id' : 'asdf',
#         'snippet' : {'trackKind' : 'thewrongkind'}
#     }]
# }

# if get_best_track_id(wrong_response):
#     print('yes')

## vigtigt:
Man kan ikke downloade captions fra videoer medmindre man ejer kontoen der har uploadet videoen eller medmindre de har givet lov til tredjeparts-downloads. 